# Count of Points in Geometries

## **actual only Polygon and Point**

### TODO .. MultiPoint, LineString and so on

#### import

In [62]:
from shapely.geometry import Point, Polygon
import geopandas as gp
import pandas as pd
from points_in_polygons import if_point_inpolygon, count_points_in_polygon, count_points_in_polygonlist

#### Test

In [45]:
# Create Point objects
p1 = Point(24.952242, 60.1696017)
p2 = Point(24.976567, 60.1612500)

p3 = Point(13.8, 57.77)

# Create a Polygon
coords = [(24.950899, 60.169158), (24.953492, 60.169158), (24.953510, 60.170104), (24.950958, 60.169990)]
poly = Polygon(coords)

In [76]:
pointlist = [p1,p2,p3]
print("List of Points -> "+str(pointlist)+"\n")
print("Check Point3 -> "+str(p3.within(poly)))
print("Check Point1 -> "+str(p1.within(poly)))
print("Count/Check List of Points in Polygon -> "+str(count_points_in_polygon(poly,pointlist)))

List of Points -> [<shapely.geometry.point.Point object at 0x7fd265b99f40>, <shapely.geometry.point.Point object at 0x7fd265b99ee0>, <shapely.geometry.point.Point object at 0x7fd265b99fa0>]

Check Point3 -> False
Check Point1 -> True
Count/Check List of Points in Polygon -> 1


In [52]:
#actual all stations where i can find
stationsvbb = gp.read_file("../results/vbb-stations.geojson")
stationsvbbsu = gp.read_file("../results/vbb-stations-s-u-bahn.geojson")
stationsnextb  = gp.read_file("../results/nextbike-stations.geojson")
stationsjelbi = gp.read_file("../results/jelbi-stations.geojson")
# Polygons from inhab-postal geojson
postalarea = gp.read_file("../results/berlin-inhabitants.geojson")

In [55]:
# add an dummy attribut public for the int list
postalarea["public"]=0

In [57]:
# transform dfx.geometry to list for the methodes
polygonlist = postalarea.geometry.tolist()
pointlist = stationsvbbsu.geometry.tolist()

In [59]:
# check the len 
print(len(pointlist))
print(len(polygonlist))

2556
198


In [61]:
#count the points in polygonlist
countliste = count_points_in_polygonlist(polygonlist,pointlist)

In [63]:
#check the count list len .. must equal the polygon list
print(len(countliste))

198


In [64]:
#concat all public stations to one dataframe
vbb_all = pd.concat([stationsvbbsu, stationsvbb])

In [65]:
#extract all points as a list
pointlist = vbb_all.geometry.tolist()

In [66]:
#counting
countliste = count_points_in_polygonlist(polygonlist,pointlist)

In [67]:
#check the len
print(len(countliste))

198


In [68]:
#add the values of public station counting to polygons 
postalarea["public"] = countliste

In [69]:
#dummy attribute for commercial counting
postalarea["com"] = 0

In [70]:
#concact two dataframes with commercial stations
com_all = pd.concat([stationsnextb, stationsjelbi])

In [69]:
#extract a list of points
pointlist = com_all.geometry.tolist()

In [70]:
#counting the points in polygons
countliste = count_points_in_polygonlist(polygonlist,pointlist)

In [72]:
# fill the attribute com with the int values 
postalarea["com"] = countliste

In [73]:
# save the dataframe with GEOJson driver
postalarea.to_file("../results/berlin-postal-with-station-count.geojson", driver='GeoJSON')